# ***<u>librariess***

In [77]:
import numpy as np
from itertools import product
from collections import defaultdict

# ***<u>Generate <u>Sudoku***

In [78]:
def generate_sudoku():

    """
        Function to generate a Sudoku grid

        
        Fill the diagonal boxes of the Sudoku grid
        Solve the Sudoku grid
        Remove elements from the grid to create a puzzle
    """
    grid = np.zeros((9, 9), dtype=int)
    fill_diagonal(grid)
    solve_sudoku(grid)
    remove_elements(grid, np.random.randint(10, 30))
    return grid


def fill_diagonal(grid):
    
    """
        Function to fill the diagonal boxes of the Sudoku grid
    """
    for i in range(0, 9, 3):
        fill_box(grid, i, i)


def fill_box(grid, row, col):

    """
        Function to fill a box with random values
    """
    for i in range(row, row + 3):
        for j in range(col, col + 3): 
            while True:
                candidate_val = np.random.randint(1, 10)
                if is_safe(grid, row, col, candidate_val):
                    grid[i][j] = candidate_val
                    break

def is_safe(grid, row, col, num):

    """
        Function to check if it is safe to place a number in a particular position
    """

    return (
        not used_in_row(grid, row, num)
        and not used_in_column(grid, col, num)
        and not used_in_box(grid, row - row % 3, col - col % 3, num)
    )


# 
def used_in_row(grid, row, num):

    """
        Function to check if a number is used in a row
    """
    for col in range(9):
        if grid[row][col] == num:
            return True 

    return False
        

def used_in_column(grid, col, num):

    """
        Function to check if a number is used in a column
    """
    for row in range(9):
        if grid[row][col] == num: 
            return True
    return False


def used_in_box(grid, box_start_row, box_start_col, num):

    """
        Function to check if a number is used in a 3x3 box
    """
    for row in range(box_start_row, box_start_row + 3):
        for col in range(box_start_col, box_start_col + 3):
            if (grid[row][col] == num):
                return True 

    return False

def find_unassigned_location(grid):

    """
        Function to find an unassigned location in the grid
    """
    for row in range(9):
        for col in range(9):
            if not(grid[row][col]):
                return (row, col)

    return (-1, -1)
    

def remove_elements(grid, num_elements):
    
    """
        Function to remove elements from the grid
    """
    while num_elements: 
        row = np.random.randint(0, 9)
        col = np.random.randint(0, 9)

        if grid[row][col]: 
            grid[row][col] = 0
            num_elements -= 1 

# ***<u>BackTracking***

In [79]:
num_backtrack_calls = 0 

def solve_sudoku(grid):
    global num_backtrack_calls

    """
        Function to solve the Sudoku grid using backtracking
    """
    row, col = find_unassigned_location(grid)
    # check that sudoko has been solved or not
    if row == -1 and col == -1: return True 

    num_backtrack_calls += 1    
    for num in range(1, 10):
        if is_safe(grid, row, col, num):
            grid[row][col] = num 
            if (solve_sudoku(grid)): 
                return True 

            grid[row][col] = 0

    return False
    

def display_grid(grid):

    """
        Function to display the Sudoku grid
    """

    for i in range(9):
        for j in range(9):
            print(grid[i][j], end=" ")
        print()

# ***<u>CSP***

In [80]:
def solve_sudoku_csp(grid):

    """
        Function to solve the Sudoku grid using Constraint Satisfaction Problem (CSP)
    """
    constraints = defaultdict(list)
    for row in range(9):
        for col in range(9):
            constraints[(row,col)]
            for z in range(9):
                if z != col:
                    constraints[(row,col)].append((row, z))
                if z != row: 
                    constraints[(row,col)].append((z, col))
            sub_i, sub_j = row // 3, col // 3
            for z in range(sub_i * 3, sub_i * 3 + 3):
                for y in range(sub_j * 3, sub_j * 3 + 3):
                    if (z != row and z != col): 
                        constraints[(row,col)].append((z, y))

    def create_domains(grid):

        """
            Function to create domains for each cell in the grid
        """
        nums = list(range(9))
        variables = product(nums, nums)
        domains = defaultdict(lambda: set(range(1, 10))) # default value for each key: {1,2,...,9}
        for (row, col) in variables:
            domains[(row, col)] # create a key with value of default
            if grid[row][col]:
                domains[(row, col)] = {grid[row][col]}
        return domains
    
            
    def is_valid_assignment(i, j, val, assignment):

        """
            Function to check if assigning a value to a cell is valid

            Check if the value is already used in the same row
            Check if the value is already used in the same column
            Check if the value is already used in the same 3x3 box
        """
        nonlocal constraints
        constraint_ij = constraints[(i, j)]
        for var in constraint_ij:
            if assignment[var] == {val}:
                return False

        return True 
        

    def find_unassigned_location(assignment):

        """
            Function to find an unassigned location in the grid
        """
        for (var, val) in assignment.items(): 
            if val == set(range(1, 10)):
                return var
        return (-1, -1)
        
    num_csp_calls = 0
    # Recursive function to solve the Sudoku grid using CSP
    def solve_csp(assignment):
        nonlocal num_csp_calls
        i, j = find_unassigned_location(assignment)
        if i == -1 and j == -1:
            return True
        num_csp_calls += 1
        # Try assigning each possible value to the unassigned location
        for val in assignment[(i, j)].copy():
            if is_valid_assignment(i, j, val, assignment):
                assignment[(i, j)] = {val}
                if solve_csp(assignment):
                    return True
                assignment[(i, j)] = set(range(1, 10))
        return False

    # Create initial domains for each cell in the grid
    domains = create_domains(grid)
    assignment = {(i, j): val for (i, j), val in domains.items()}

    # Solve the Sudoku grid using CSPFr
    if solve_csp(assignment):
        solved_grid = [[assignment[(i, j)] for j in range(9)] for i in range(9)]
        print("num_csp_calls: ", num_csp_calls)
        return solved_grid
    else:
        return None

# ***<u>Show <u>Result***

In [81]:
# Function to initialize the Sudoku grid
def initializing_grid():
    print("\nInitial Sudoku\n")
    sudoku_grid = generate_sudoku()
    for i in sudoku_grid:
        for j in i:
            print(j, end=' ')
        print("")
    print(end='\n\n\n\n\n')
    return sudoku_grid


# Function to solve the Sudoku grid using backtracking
def backtracking_answer(sudoku_grid):
    print("Back Tracking Answer:\n\n")
    print("num_backtrack_calls: ", num_backtrack_calls)
    if solve_sudoku(sudoku_grid):
        print("Sudoku solved successfully:")
        display_grid(sudoku_grid)
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')


# Function to solve the Sudoku grid using CSP
def csp_answer(sudoku_grid):
    print("CSP Answer:\n")
    solved_grid = solve_sudoku_csp(sudoku_grid)
    if solved_grid is not None:
        print("Sudoku solved successfully:")
        for row in solved_grid:
            for r1 in row:  
                for r2 in r1:
                    print(r2, end=' ')
            print()
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')

In [82]:
# Generate and display the initial Sudoku grid
generate_sudoko = initializing_grid()
generate_sudoko_copy = np.copy(generate_sudoko)

# Solve the Sudoku grid using backtracking
backtracking_answer(generate_sudoko_copy)

# Solve the Sudoku grid using CSP
csp_answer(generate_sudoko)


Initial Sudoku

0 5 1 0 2 6 0 9 8 
0 8 0 0 4 7 3 0 5 
6 0 7 8 9 0 2 4 1 
2 4 5 0 0 1 0 0 0 
1 9 0 0 3 4 0 0 7 
7 6 3 0 0 9 4 1 2 
3 1 6 4 5 2 0 7 9 
8 7 4 9 1 3 0 2 6 
5 0 9 7 0 8 0 3 4 





Back Tracking Answer:


num_backtrack_calls:  119
Sudoku solved successfully:
4 5 1 3 2 6 7 9 8 
9 8 2 1 4 7 3 6 5 
6 3 7 8 9 5 2 4 1 
2 4 5 6 7 1 9 8 3 
1 9 8 2 3 4 6 5 7 
7 6 3 5 8 9 4 1 2 
3 1 6 4 5 2 8 7 9 
8 7 4 9 1 3 5 2 6 
5 2 9 7 6 8 1 3 4 





CSP Answer:

num_csp_calls:  27
Sudoku solved successfully:
4 5 1 3 2 6 7 9 8 
9 8 2 1 4 7 3 6 5 
6 3 7 8 9 5 2 4 1 
2 4 5 6 7 1 9 8 3 
1 9 8 2 3 4 6 5 7 
7 6 3 5 8 9 4 1 2 
3 1 6 4 5 2 8 7 9 
8 7 4 9 1 3 5 2 6 
5 2 9 7 6 8 1 3 4 





